## 서울 요식업 데이터 EDA

In [4]:
seoul = spark.read.csv('data/seoulfood-full.csv', header=True, inferSchema=True, encoding='euc-kr')
seoul.printSchema()

root
 |-- 개방자치단체코드: integer (nullable = true)
 |-- 관리번호: string (nullable = true)
 |-- 인허가일자: integer (nullable = true)
 |-- 인허가취소일자: string (nullable = true)
 |-- 영업상태코드: integer (nullable = true)
 |-- 영업상태명: string (nullable = true)
 |-- 상세영업상태코드: integer (nullable = true)
 |-- 상세영업상태명: string (nullable = true)
 |-- 폐업일자: integer (nullable = true)
 |-- 휴업시작일자: string (nullable = true)
 |-- 휴업종료일자: string (nullable = true)
 |-- 재개업일자: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- 소재지면적: double (nullable = true)
 |-- 소재지우편번호: string (nullable = true)
 |-- 지번주소: string (nullable = true)
 |-- 도로명주소: string (nullable = true)
 |-- 도로명우편번호: string (nullable = true)
 |-- 사업장명: string (nullable = true)
 |-- 최종수정일자: string (nullable = true)
 |-- 데이터갱신구분: string (nullable = true)
 |-- 데이터갱신일자: string (nullable = true)
 |-- 업태구분명: string (nullable = true)
 |-- 좌표정보(X): double (nullable = true)
 |-- 좌표정보(Y): string (nullable = true)
 |-- 위생업태명: string (nullable = true)
 |-- 남성

In [6]:
seoul.columns

['개방자치단체코드',
 '관리번호',
 '인허가일자',
 '인허가취소일자',
 '영업상태코드',
 '영업상태명',
 '상세영업상태코드',
 '상세영업상태명',
 '폐업일자',
 '휴업시작일자',
 '휴업종료일자',
 '재개업일자',
 '전화번호',
 '소재지면적',
 '소재지우편번호',
 '지번주소',
 '도로명주소',
 '도로명우편번호',
 '사업장명',
 '최종수정일자',
 '데이터갱신구분',
 '데이터갱신일자',
 '업태구분명',
 '좌표정보(X)',
 '좌표정보(Y)',
 '위생업태명',
 '남성종사자수',
 '여성종사자수',
 '영업장주변구분명',
 '등급구분명',
 '급수시설구분명',
 '총인원',
 '본사종업원수',
 '공장사무직종업원수',
 '공장판매직종업원수',
 '공장생산직종업원수',
 '건물소유구분명',
 '보증액',
 '월세액',
 '다중이용업소여부',
 '시설총규모',
 '전통업소지정번호',
 '전통업소주된음식',
 '홈페이지']

In [24]:
# 쓸 컬럼명들만 색출해냄
sf = seoul.select('인허가일자','영업상태명','도로명주소','사업장명','업태구분명','좌표정보(X)','좌표정보(Y)')
sf.columns

['인허가일자', '영업상태명', '도로명주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)']

In [25]:
sf.show(5)

+----------+----------+---------------------------------+---------------------+------------+----------------+----------------+
|인허가일자|영업상태명|                       도로명주소|             사업장명|  업태구분명|     좌표정보(X)|     좌표정보(Y)|
+----------+----------+---------------------------------+---------------------+------------+----------------+----------------+
|  20201229| 영업/정상|  서울특별시 중구 청계천로 400...|  통불 이마트청계천점|일반조리판매|201823.908977364|452076.818664092|
|  20200713| 영업/정상|  서울특별시 성북구 길음로 103...|  씨유 길음푸르지오점|      편의점|201517.385329678|456504.561867278|
|  20200713| 영업/정상|서울특별시 송파구 위례성대로 6...|스타벅스 몽촌토성역점|      커피숍|209929.574783236|446085.121305444|
|  20200713| 영업/정상|서울특별시 동대문구 전농로16길...|         Min's coffee|      커피숍|205263.826418987|452869.992526477|
|  20200713| 영업/정상| 서울특별시 강북구 삼양로87길 ...|             탑브릭스|일반조리판매|201292.010942271|459312.320007983|
+----------+----------+---------------------------------+---------------------+------------+----------------+----------------+
only showing top 5 row

In [32]:
import pyspark.sql.functions as F

In [36]:
### 음식점 종류 요약
sf.select('업태구분명').where(F.length(sf.업태구분명) <= 15)\
    .groupBy('업태구분명').count()\
    .orderBy('count', ascending=False).show(30)

+---------------+-----+
|     업태구분명|count|
+---------------+-----+
|         커피숍|28646|
|   일반조리판매|19076|
|           다방|17717|
|기타 휴게음식점|15778|
|         편의점| 9761|
|         과자점| 8678|
|     패스트푸드| 7853|
|         백화점| 2791|
|       푸드트럭|  974|
|     아이스크림|  464|
|     철도역구내|  451|
|       전통찻집|  351|
|         떡카페|  147|
|       키즈카페|  137|
|       관광호텔|   80|
|         유원지|   60|
|           극장|   22|
|           공항|   18|
|       고속도로|    9|
|           기타|    2|
|           한식|    2|
|         룸살롱|    1|
|         ?蓉?점|    1|
|   김밥(도시락)|    1|
|      호프/통닭|    1|
|   ?球北떳?판매|    1|
|       단란주점|    1|
+---------------+-----+



In [37]:
### 영업상태 요약
sf.select('영업상태명').groupBy('영업상태명').count().show()

+----------+-----+
|영업상태명|count|
+----------+-----+
| 영업/정상|37347|
|      폐업|75687|
+----------+-----+



In [39]:
### 커피숍 종류는 몇 개나? - 1)
sf.select('업태구분명', '사업장명').where(sf.업태구분명 == '커피숍')\
    .groupBy('사업장명').count().orderBy('count', ascending=False).show()

+-----------------------+-----+
|               사업장명|count|
+-----------------------+-----+
|                 마이빈|   59|
|                 이디야|   58|
|             요거프레소|   55|
|               커피마마|   47|
|             이디야커피|   47|
|               커피나무|   39|
|               카페베네|   39|
|               카페제이|   34|
|                 카페큐|   33|
|               커피크루|   31|
|               토프레소|   29|
|               탐앤탐스|   28|
|             할리스커피|   27|
|           투썸플레이스|   26|
|               커피베이|   24|
|                카페 큐|   24|
|(주)커피빈코리아 행사점|   23|
|                 커피집|   23|
|                   쥬씨|   22|
|                 더카페|   21|
+-----------------------+-----+
only showing top 20 rows



In [42]:
### 커피숍 종류는 몇 개나? - 2)
sf.select('업태구분명', '사업장명').where(sf.업태구분명 == '커피숍')\
    .groupBy(F.substring(sf.사업장명,0,4).alias('사업장명')).count()\
    .orderBy('count', ascending=False).show()

+--------+-----+
|사업장명|count|
+--------+-----+
|스타벅스|  556|
|투썸플레|  283|
|이디야커|  238|
|  (주)커|  226|
|커피베이|  220|
| 이디야 |  203|
|메가엠지|  177|
|요거프레|  167|
|엔제리너|  164|
|탐앤탐스|  160|
|할리스커|  135|
|카페베네|  133|
|컴포즈커|  109|
|파스쿠찌|  102|
|커피나무|   99|
|    cafe|   95|
|    Cafe|   95|
|토프레소|   91|
| 빽다방 |   91|
|커피마마|   73|
+--------+-----+
only showing top 20 rows



In [45]:
### 커피숍 종류는 몇 개나?- 3)
sf.select('업태구분명', '사업장명').where(sf.업태구분명 == '커피숍')\
    .groupBy(F.split(sf.사업장명, ' ')[0].alias('사업장명')).count()\
    .orderBy('count', ascending=False).show(45)

+----------------+-----+
|        사업장명|count|
+----------------+-----+
|            카페|  716|
|        스타벅스|  325|
|          이디야|  261|
|    투썸플레이스|  224|
|(주)커피빈코리아|  169|
|        커피베이|  157|
|  메가엠지씨커피|  154|
|      이디야커피|  144|
|      요거프레소|  131|
|        탐앤탐스|  117|
|            공차|  116|
|      할리스커피|  101|
|            커피|   99|
|          빽다방|   97|
|      엔제리너스|   94|
|      컴포즈커피|   89|
|        카페베네|   88|
|            Cafe|   84|
|            쥬씨|   82|
|    스타벅스커피|   78|
|            cafe|   71|
|        토프레소|   71|
|              더|   71|
|        파스쿠찌|   70|
|          할리스|   67|
|        커피나무|   65|
|        커피마마|   61|
|        주식회사|   60|
|          마이빈|   59|
|          매머드|   49|
|          커피에|   48|
|      셀렉토커피|   48|
|          더벤티|   45|
|    커피에반하다|   43|
|매머드익스프레스|   42|
|            까페|   40|
|          더카페|   37|
|      세븐일레븐|   36|
|        만랩커피|   35|
|          커피니|   35|
|  엔제리너스커피|   35|
|            CAFE|   35|
|        카페제이|   34|
|          카페큐|   

In [55]:
### 구별 스타벅스는 몇 개나? 
sf.select('업태구분명', '도로명주소', '사업장명')\
    .where(sf.사업장명.like('%스타벅스%'))\
    .groupBy(F.split(sf.도로명주소, ' ')[1].alias('구별'))\
    .count().orderBy('count', ascending=False).show(10)

+--------+-----+
|    구별|count|
+--------+-----+
|  강남구|  100|
|    중구|   64|
|    null|   59|
|  서초구|   55|
|영등포구|   44|
|  종로구|   44|
|  마포구|   36|
|  송파구|   34|
|서대문구|   24|
|  용산구|   24|
+--------+-----+
only showing top 10 rows



In [54]:
### 구별 스타벅스는 몇 개나? 
sf.select('업태구분명', '도로명주소', '사업장명')\
    .where(sf.사업장명.like('%스타벅스%'))\
    .where(sf.도로명주소 != '')\
    .groupBy(F.split(sf.도로명주소, ' ')[1].alias('구별'))\
    .count().orderBy('count', ascending=False).show(10)

+--------+-----+
|    구별|count|
+--------+-----+
|  강남구|  100|
|    중구|   64|
|  서초구|   55|
|영등포구|   44|
|  종로구|   44|
|  마포구|   36|
|  송파구|   34|
|  용산구|   24|
|서대문구|   24|
|  강서구|   21|
+--------+-----+
only showing top 10 rows

